### Load data

In [11]:
import pandas as pd
books_data = pd.read_csv('data/books_data.csv')
books_ratings = pd.read_csv('data/books_rating.csv')

In [12]:
# Titles
books_data_1 = books_data.drop(['image', 'previewLink', 'infoLink', 'publisher', 'ratingsCount'], axis=1, inplace=False)
books_ratings_1 = books_ratings.drop(['Id','Price', 'profileName', 'review/time'], axis=1, inplace=False)

books_data_1.rename(columns={'Title': 'title','publishedDate': 'published_date'}, inplace=True)
books_ratings_1.rename(columns={'Title': 'title','User_id':'user_id',  'review/score': 'score','review/helpfulness': 'helpfulness', 'review/text': 'text', 'review/summary': 'summary'}, inplace=True)

print(list(books_data_1.columns))
print(list(books_ratings_1.columns))

['title', 'description', 'authors', 'published_date', 'categories']
['title', 'user_id', 'helpfulness', 'score', 'summary', 'text']


In [15]:
books_ratings_1['title'] = books_ratings_1['title'].map(lambda s: s.lower() if type(s) == str else s)
books_ratings_1['title'] = books_ratings_1['title'].replace(r"\(.*\)","", regex=True)

In [17]:
# load book data for books in book ratings
ratings_titles = books_ratings_1['title'].unique()
books_data_1 = books_data_1[books_data_1['title'].isin(ratings_titles)]

In [18]:
x = books_ratings_1.groupby('user_id').count()['score'] > 5
considerable_users = x[x].index
books_ratings_1 = books_ratings_1[books_ratings_1['user_id'].isin(considerable_users)]
#
# print(books_ratings_1.sample(5))

books_ratings_1['user_id'].value_counts()

user_id
A14OJS0VWMOSWO    5795
AFVQZQ8PW0L       3606
A1D2C0WDCSHUWZ    3146
AHD101501WCN1     1995
A1X8VZWTOG8IS6    1804
                  ... 
A2LEUNJRMFJCD2       6
A2JF696S5V7ZLV       6
A9Z94SVPIFAQH        6
A1B0BCGTNNHT7X       6
A880CKJM9H94U        6
Name: count, Length: 65571, dtype: int64

In [23]:
# extract all books rated by A14OJS0VWMOSWO

books_ratings_1[books_ratings_1['user_id'] == 'A14OJS0VWMOSWO']['title'].unique()

array(['dr. seuss: american icon',
       "dealing with disappointment: helping kids cope when things don't go their way",
       "communicating with orcas - the whales' perspective", ...,
       'the encyclopedia of genetic disorders and birth defects ',
       'trivia for smarties', 'sweep dreams'], dtype=object)

In [100]:
n_ratings = books_ratings_1.groupby('title').count()['score']>=10
famous_books = n_ratings[n_ratings].index
books_ratings_1 = books_ratings_1[books_ratings_1['title'].isin(famous_books)]

n_ratings_1 = books_ratings_1['title'].value_counts().rename("n_ratings")

books_data_1 = books_data_1.merge(n_ratings_1, on='title', how='inner')
#print(books_data_1.columns)

In [101]:
avg_rating = books_ratings_1.groupby('title')["score"].mean().rename("avg_rating")
books_data_1 = books_data_1.merge(avg_rating, on='title', how='inner')
#print(books_data_1.columns)


In [102]:
# print(books_data_1.sample(1))

In [103]:
def get_n_helpful(x):
    if isinstance(x, str) and '/' in x:
        num, denom = x.split('/')
        return int(denom)
    else:
        return x

def replace_fraction(x):
    if isinstance(x, str) and '/' in x:
        num, denom = x.split('/')
        if denom == '0':
            return 0
        else:
            return int(num) / int(denom)
    else:
        return x

books_ratings_1['helpfulness_count'] = books_ratings_1['helpfulness'].apply(get_n_helpful)
books_ratings_1['helpfulness_pct'] = books_ratings_1['helpfulness'].apply(replace_fraction)

#print(books_ratings_1.sample(2))

In [104]:
#print(books_data_1.columns)

In [105]:
#def extract_year(x):
#    year1 = pd.to_datetime(x, format='mixed').year
##    print(year1)
#    return year1
    
#books_data_1['published_date'] = books_data_1['published_date'].apply(extract_year)
#p#rint(books_data_1.sample(5))


In [106]:
import os
if not os.path.exists('clean_data'):
    os.makedirs('clean_data')

# save clean data
books_data_1.to_csv('clean_data/books_data_clean.csv')
books_ratings_1.to_csv('clean_data/books_rating_clean.csv')

print("First we had {} books and {} ratings".format(len(books_data), len(books_ratings)))
print("Now we have {} books and {} ratings".format(len(books_data_1), len(books_ratings_1)))

First we had 212404 books and 3000000 ratings
Now we have 13188 books and 834804 ratings
